<a href="https://colab.research.google.com/github/mjgpinheiro/Econophysics/blob/main/Punctuate_eq_stockmarket_MT5_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import xmapi  # import the XMTradingAPI library
import pandas as pd
import numpy as np
import MetaTrader5 as mt5

# Connect to XM broker API
api_key = 'your_api_key_here'
api_secret = 'your_api_secret_here'
api = xmapi.XMTradingAPI(api_key, api_secret)

# Define parameters
symbol = 'US100'
lot_size = 0.1
stop_loss = 0.01
take_profit = 0.02
threshold = 0.5

# Connect to MT5 platform and load historical data
mt5.initialize()
nasdaq_data = pd.DataFrame(mt5.copy_rates_from_pos(symbol, mt5.TIMEFRAME_D1, 0, 1000))
nasdaq_data.drop(columns=['spread', 'real_volume'], inplace=True)
nasdaq_data['date'] = pd.to_datetime(nasdaq_data['time'], unit='s')
nasdaq_data.set_index('date', inplace=True)

# Define function to execute trades
def trade(strategy):
    current_position = api.get_position(symbol)
    if strategy == 'buy':
        if current_position is None:
            api.create_market_buy_order(symbol, lot_size, stop_loss, take_profit)
        elif current_position.type == 'sell':
            api.close_position(symbol)
            api.create_market_buy_order(symbol, lot_size, stop_loss, take_profit)
    elif strategy == 'sell':
        if current_position is None:
            api.create_market_sell_order(symbol, lot_size, stop_loss, take_profit)
        elif current_position.type == 'buy':
            api.close_position(symbol)
            api.create_market_sell_order(symbol, lot_size, stop_loss, take_profit)

# Calculate moving average and stability
nasdaq_data['ma200'] = nasdaq_data['close'].rolling(window=200).mean()
nasdaq_data['diff'] = nasdaq_data['close'] - nasdaq_data['ma200']
nasdaq_data['std'] = nasdaq_data['diff'].rolling(window=20).std()
nasdaq_data['stable'] = np.where(nasdaq_data['std'] < threshold, True, False)

# Iterate over each day and execute trades according to strategy
for i in range(1, len(nasdaq_data)):
    prev_ma200 = nasdaq_data.iloc[i-1]['ma200']
    curr_ma200 = nasdaq_data.iloc[i]['ma200']
    prev_stable = nasdaq_data.iloc[i-1]['stable']
    curr_stable = nasdaq_data.iloc[i]['stable']
    prev_close = nasdaq_data.iloc[i-1]['close']
    curr_close = nasdaq_data.iloc[i]['close']
    
    # Determine current strategy based on moving average and stability
    if prev_close < prev_ma200 and curr_close > curr_ma200 and curr_stable:
        strategy = 'buy'
    elif prev_close > prev_ma200 and curr_close < curr_ma200 and curr_stable:
        strategy = 'sell'
    else:
        strategy = None
        
    # Execute trade if strategy is not None
    if strategy is not None:
        trade(strategy)
        
# Disconnect from XM broker API and MT5
api.logout()
mt5.shutdown()
